In [9]:
import numpy as np
from onnx import helper

In [8]:
dimension = np.array([1, 3, 5, 5, 10])
vals = np.prod(dimension)
arr = np.arange(vals).reshape(dimension)
arr.reshape(1, -1, 10).shape

(1, 75, 10)

In [ ]:
onnx.helper.make_node(
    'Reshape',
    inputs=['date'],
    outputs=['data_reshaped']
)

In [ ]:
onnx_fpath = [Path to .onnx model]
onnx_model = onnx.load(onnx_fpath)

# make constant tensors
score_threshold = helper.make_tensor("score_threshold", TensorProto.FLOAT, [1], [0.9])
iou_threshold = helper.make_tensor("iou_threshold", TensorProto.FLOAT, [1], [0.3])
max_output_boxes_per_class = onnx.helper.make_tensor("max_output_boxes_per_class", TensorProto.INT64, [1], [200])

# create the NMS node
inputs=['bboxes', 'scores', 'max_output_boxes_per_class', 'iou_threshold', 'score_threshold',]
outputs = ["selected_indices"]
nms_node = onnx.helper.make_node(
    'NonMaxSuppression',
    inputs,
    ["selected_indices"],
)

# add to the list of graph nodes
graph.node.append(nms_node)

# append to the output (now the outputs would be scores, bboxes, selected_indices)
output_value_info = onnx.helper.make_tensor_value_info("selected_indices", TensorProto.INT64, shape=[])
graph.output.append(output_value_info)

# add to initializers - without this, onnx will not know where these came from, and complain that 
# they're neither outputs of other nodes, nor inputs. As initializers, however, they are treated 
# as constants needed for the NMS op
graph.initializer.append(score_threshold)
graph.initializer.append(iou_threshold)
graph.initializer.append(max_output_boxes_per_class)

# check that it works and re-save
onnx.checker.check_model(onnx_model)
onnx.save(onnx_model, onnx_fpath)